In [1]:
import json
import numpy as np
from input_data import json_string_1, xml_file_2, json_string_2
from xml_generator import json_to_flattened_dict, generate_flows_based_on_elements, bpmn_to_json
from data_structure_methods import dump_positions, init_element_positions, assign_shift_and_depth
from data_structures import Node, Edge

In [2]:
import re
import ast
import json

parsed_jsons = []

def clean_line(line):
    # Remove ObjectId(...) and datetime.datetime(...) safely
    line = re.sub(r"ObjectId\('.*?'\)", '"ObjectId"', line)
    line = re.sub(r"datetime\.datetime\([^\)]+\)", '"datetime"', line)
    return line

with open("out.txt", "r") as file:
    for line in file:
        try:
            cleaned_line = clean_line(line)

            # Safely parse the outer line to a Python dictionary
            line_dict = ast.literal_eval(cleaned_line)

            # Get the 'content' string
            content_raw = line_dict.get('content')
            if content_raw:
                # Parse the inner string content into a dictionary
                content_dict = ast.literal_eval(content_raw)
                parsed_jsons.append(content_dict)

        except Exception as e:
            print(f"Failed to parse line: {e}")

In [3]:
import json
import pydot
import re

for process_i, json_process in enumerate(parsed_jsons):
    print("processing entry ",process_i)
    json_data = json.dumps(json_process, indent=4)
    parsed_json_input = json.loads(json_data)
    
    for elem in parsed_json_input["elements"]:
        elem["outgoing_flows"] = []
        elem["incoming_flows"] = []
    
    parsed_json = generate_flows_based_on_elements(parsed_json_input)
    elements_as_data_struc = assign_shift_and_depth(parsed_json)

    elem_pos = init_element_positions(elements_as_data_struc)
    from xml_generator import xml_position_appender, generate_top_bpmn_xml
    top_xml_part = generate_top_bpmn_xml(elem_pos)
    xml_position_appender(elem_pos, top_xml_part, f"json_process_v12_{process_i}")

processing entry  0
{'elements': [{'id': 'Event_1fgdyvq', 'type': 'startEvent', 'label': 'probable recourse detected', 'incoming': [], 'outgoing': ['flow_Event_1fgdyvq_Task_0iirfhd']}, {'id': 'Task_0iirfhd', 'type': 'task', 'label': 'get information', 'incoming': ['flow_Event_1fgdyvq_Task_0iirfhd'], 'outgoing': ['flow_Task_0iirfhd_Task_0iirfhd1']}, {'id': 'Task_0iirfhd1', 'type': 'task', 'label': 'check case', 'incoming': ['flow_Task_0iirfhd_Task_0iirfhd1'], 'outgoing': ['flow_Task_0iirfhd1_ExclusiveGateway_092mc05']}, {'id': 'ExclusiveGateway_092mc05', 'type': 'exclusiveGateway', 'label': 'recourse possible?', 'incoming': ['flow_Task_0iirfhd1_ExclusiveGateway_092mc05'], 'outgoing': ['flow_ExclusiveGateway_092mc05_Task_02fdytg', 'flow_ExclusiveGateway_092mc05_Task_04aofbe']}, {'id': 'Task_02fdytg', 'type': 'task', 'label': 'send request for payment', 'incoming': ['flow_ExclusiveGateway_092mc05_Task_02fdytg'], 'outgoing': ['flow_Task_02fdytg_Task_12lthpj']}, {'id': 'Task_12lthpj', 'type

NameError: name 'Node' is not defined